This is the notebook where I take pitcher starting performance, and create a df of "previous" performance before each start, for each pitcher.

In [45]:
import pandas as pd
import warnings
import numpy as np
warnings.simplefilter(action='ignore', category=FutureWarning)


In [47]:
df = pd.read_csv('pitcher_data.csv')
df


,Date,Opp,Dec,DR,H,ER,BB,SO,HR,HBP,FIP,Outs,points,QS,pitcher,code
0,2023-04-05,ATL,-1,5,9,5,1,6,1,0,2.72,18,1,0.0,Miles Mikolas,mikolmi01
1,2023-04-11,COL,0,5,10,6,2,3,3,0,5.63,15,-6,0.0,Miles Mikolas,mikolmi01
2,2023-04-16,PIT,0,4,7,2,2,4,0,0,4.86,17,8,0.0,Miles Mikolas,mikolmi01
3,2023-04-22,SEA,0,5,5,3,2,4,2,0,5.47,16,7,0.0,Miles Mikolas,mikolmi01
4,2023-04-27,SFG,1,4,4,0,2,6,0,1,4.93,19,21,1.0,Miles Mikolas,mikolmi01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,2023-08-12,NYY,1,5,5,1,2,10,0,0,3.78,27,31,1.0,Sandy Alcantara,alcansa01
198,2023-08-18,LAD,1,5,7,3,1,6,3,0,3.95,18,13,1.0,Sandy Alcantara,alcansa01
199,2023-08-23,SDP,-1,4,7,4,3,3,1,0,4.01,20,2,0.0,Sandy Alcantara,alcansa01
200,2023-08-29,TBR,-1,5,7,4,2,4,0,1,4.00,17,0,0.0,Sandy Alcantara,alcansa01


In [48]:
# This is me getting a df of how the given pitcher performed
# in the outing immediately before the given date
df_last = pd.DataFrame()

for i in range(len(df)):
    # For a given row i, extract the code and date for that row
    date = df.loc[i,:].Date
    code = df.loc[i,:].code

    # Given a code/date, extract the previous game data
    df_last1 = df.loc[(df.Date < date) & (df.code == code)].sort_values('Date').tail(1)
    
    df_last1 = df_last1.drop(columns = ['Date', 'Opp','pitcher','code'])

    # Get the average of those 5 rows, find the averages of averageable, transpose it from series to row
    df_last1_avg = pd.DataFrame(df_last1.mean(axis=0)).transpose()

    # Rename cols to indicate
    df_last1_avg.columns = [f"{col}_1" for col in df_last1_avg.columns]

    # Assign it to the date and code we're working with
    df_last1_avg['Date'] = date
    df_last1_avg['code'] = code

    # Append into the last few games df
    df_last = df_last.append(df_last1_avg)

# Rest the index so that we can .loc through it
df_last = df_last.reset_index(drop=True)

# Put in the performance buckets
for row in range(1,len(df_last)):
    if df_last.loc[row,'points_1']<=-10: df_last.loc[row,'bucket_1'] = '.<=-10'
    elif df_last.loc[row,'points_1']<=0: df_last.loc[row,'bucket_1'] = '-10<.<=0'
    elif df_last.loc[row,'points_1']<=10: df_last.loc[row,'bucket_1'] = '0<.<=10'
    elif df_last.loc[row,'points_1']<=20: df_last.loc[row,'bucket_1'] = '10<.<=20'
    elif df_last.loc[row,'points_1']<=30: df_last.loc[row,'bucket_1'] = '20<.<=30'
    elif df_last.loc[row,'points_1']>30: df_last.loc[row,'bucket_1'] = '30<.'
    else: df_last.loc[row,'bucket_1'] = np.nan

df_last

,Dec_1,DR_1,H_1,ER_1,BB_1,SO_1,HR_1,HBP_1,FIP_1,Outs_1,points_1,QS_1,Date,code,bucket_1
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-04-05,mikolmi01,NaN
1,-1.0,5.0,9.0,5.0,1.0,6.0,1.0,0.0,2.72,18.0,1.0,0.0,2023-04-11,mikolmi01,0<.<=10
2,0.0,5.0,10.0,6.0,2.0,3.0,3.0,0.0,5.63,15.0,-6.0,0.0,2023-04-16,mikolmi01,-10<.<=0
3,0.0,4.0,7.0,2.0,2.0,4.0,0.0,0.0,4.86,17.0,8.0,0.0,2023-04-22,mikolmi01,0<.<=10
4,0.0,5.0,5.0,3.0,2.0,4.0,2.0,0.0,5.47,16.0,7.0,0.0,2023-04-27,mikolmi01,0<.<=10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,-1.0,4.0,6.0,4.0,0.0,7.0,3.0,0.0,3.91,18.0,8.0,0.0,2023-08-12,alcansa01,0<.<=10
198,1.0,5.0,5.0,1.0,2.0,10.0,0.0,0.0,3.78,27.0,31.0,1.0,2023-08-18,alcansa01,30<.
199,1.0,5.0,7.0,3.0,1.0,6.0,3.0,0.0,3.95,18.0,13.0,1.0,2023-08-23,alcansa01,10<.<=20
200,-1.0,4.0,7.0,4.0,3.0,3.0,1.0,0.0,4.01,20.0,2.0,0.0,2023-08-29,alcansa01,0<.<=10


In [50]:
# Variable to determine the length of our rolling average
N=5

df_lastfew = pd.DataFrame()

for i in range(len(df)):
    # For a given row i, extract the code and date for that row
    date = df.loc[i,:].Date
    code = df.loc[i,:].code

    # Given a code/date, extract the previous N games of data
    df_lastN = df.loc[(df.Date < date) & (df.code == code)].sort_values('Date').tail(N)
    
    df_lastN = df_lastN.drop(columns = ['Date', 'Opp','pitcher','code'])

    # Get the average of those N rows, find the averages of averageable series, transpose it from series to row
    df_lastN_avg = pd.DataFrame(df_lastN.mean(axis=0)).transpose()

    # Rename cols to indicate what it's the average of, and over how many games
    df_lastN_avg.columns = [f"{col}_{N}" for col in df_lastN_avg.columns]

    # Assign it to the date and code we're working with
    df_lastN_avg['Date'] = date
    df_lastN_avg['code'] = code

    # Append into the last few games df
    df_lastfew = df_lastfew.append(df_lastN_avg)

# Rest the index so that we can .loc through it
df_lastfew = df_lastfew.reset_index(drop=True)

# Put in the performance buckets
for row in range(1,len(df_lastfew)):
    if df_lastfew.loc[row,f'points_{N}']<=-10: df_lastfew.loc[row,f'bucket_{N}'] = '.<=-10'
    elif df_lastfew.loc[row,f'points_{N}']<=0: df_lastfew.loc[row,f'bucket_{N}'] = '-10<.<=0'
    elif df_lastfew.loc[row,f'points_{N}']<=10: df_lastfew.loc[row,f'bucket_{N}'] = '0<.<=10'
    elif df_lastfew.loc[row,f'points_{N}']<=20: df_lastfew.loc[row,f'bucket_{N}'] = '10<.<=20'
    elif df_lastfew.loc[row,f'points_{N}']<=30: df_lastfew.loc[row,f'bucket_{N}'] = '20<.<=30'
    elif df_lastfew.loc[row,f'points_{N}']>30: df_lastfew.loc[row,f'bucket_{N}'] = '30<.'
    else: df_lastfew.loc[row,f'bucket_{N}'] = np.nan

df_lastfew

,Dec_5,DR_5,H_5,ER_5,BB_5,SO_5,HR_5,HBP_5,FIP_5,Outs_5,points_5,QS_5,Date,code,bucket_5
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-04-05,mikolmi01,NaN
1,-1.000000,5.000000,9.000000,5.000000,1.000000,6.000000,1.000000,0.0,2.720000,18.000000,1.0,0.0,2023-04-11,mikolmi01,0<.<=10
2,-0.500000,5.000000,9.500000,5.500000,1.500000,4.500000,2.000000,0.0,4.175000,16.500000,-2.5,0.0,2023-04-16,mikolmi01,-10<.<=0
3,-0.333333,4.666667,8.666667,4.333333,1.666667,4.333333,1.333333,0.0,4.403333,16.666667,1.0,0.0,2023-04-22,mikolmi01,0<.<=10
4,-0.250000,4.750000,7.750000,4.000000,1.750000,4.250000,1.500000,0.0,4.670000,16.500000,2.5,0.0,2023-04-27,mikolmi01,0<.<=10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,-0.400000,5.000000,6.200000,2.200000,1.000000,6.200000,1.200000,0.0,3.890000,21.000000,14.4,0.6,2023-08-12,alcansa01,10<.<=20
198,0.000000,4.800000,5.600000,2.000000,1.200000,7.200000,0.800000,0.0,3.852000,22.800000,19.2,0.6,2023-08-18,alcansa01,10<.<=20
199,0.400000,5.000000,5.400000,1.800000,1.000000,7.000000,1.200000,0.0,3.848000,22.800000,21.0,0.8,2023-08-23,alcansa01,20<.<=30
200,0.000000,4.600000,5.800000,2.400000,1.400000,6.200000,1.400000,0.0,3.882000,21.400000,15.6,0.6,2023-08-29,alcansa01,10<.<=20


In [51]:
df_cummulative = pd.DataFrame()

for i in range(len(df)):
    # For a given row i, extract the code and date for that row
    date = df.loc[i,:].Date
    code = df.loc[i,:].code

    # Given a code/date, extract the previous game data
    df_cummu = df.loc[(df.Date < date) & (df.code == code)].sort_values('Date')
    
    df_cummu = df_cummu.drop(columns = ['Date', 'Opp','pitcher','code'])

    # Get the average of all rows, find the averages of averageable, transpose it from series to row
    df_cummu_avg = pd.DataFrame(df_cummu.mean(axis=0)).transpose()

    # Rename cols to indicate
    df_cummu_avg.columns = [f"{col}_all" for col in df_cummu_avg.columns]

    # Assign it to the date and code we're working with
    df_cummu_avg['Date'] = date
    df_cummu_avg['code'] = code

    # Append into the last few games df
    df_cummulative = df_cummulative.append(df_cummu_avg)

# Rest the index so that we can .loc through it
df_cummulative = df_cummulative.reset_index(drop=True)

# Put in the performance buckets
for row in range(1,len(df_cummulative)):
    if df_cummulative.loc[row,'points_all']<=-10: df_cummulative.loc[row,'bucket_all'] = '.<=-10'
    elif df_cummulative.loc[row,'points_all']<=0: df_cummulative.loc[row,'bucket_all'] = '-10<.<=0'
    elif df_cummulative.loc[row,'points_all']<=10: df_cummulative.loc[row,'bucket_all'] = '0<.<=10'
    elif df_cummulative.loc[row,'points_all']<=20: df_cummulative.loc[row,'bucket_all'] = '10<.<=20'
    elif df_cummulative.loc[row,'points_all']<=30: df_cummulative.loc[row,'bucket_all'] = '20<.<=30'
    elif df_cummulative.loc[row,'points_all']>30: df_cummulative.loc[row,'bucket_all'] = '30<.'
    else: df_cummulative.loc[row,'bucket_all'] = np.nan

df_cummulative


,Dec_all,DR_all,H_all,ER_all,BB_all,SO_all,HR_all,HBP_all,FIP_all,Outs_all,points_all,QS_all,Date,code,bucket_all
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-04-05,mikolmi01,NaN
1,-1.000000,5.000000,9.000000,5.000000,1.000000,6.000000,1.000000,0.000000,2.720000,18.000000,1.000000,0.000000,2023-04-11,mikolmi01,0<.<=10
2,-0.500000,5.000000,9.500000,5.500000,1.500000,4.500000,2.000000,0.000000,4.175000,16.500000,-2.500000,0.000000,2023-04-16,mikolmi01,-10<.<=0
3,-0.333333,4.666667,8.666667,4.333333,1.666667,4.333333,1.333333,0.000000,4.403333,16.666667,1.000000,0.000000,2023-04-22,mikolmi01,0<.<=10
4,-0.250000,4.750000,7.750000,4.000000,1.750000,4.250000,1.500000,0.000000,4.670000,16.500000,2.500000,0.000000,2023-04-27,mikolmi01,0<.<=10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,-0.272727,4.863636,6.272727,3.090909,1.545455,5.590909,0.772727,0.181818,3.754545,19.590909,10.181818,0.454545,2023-08-12,alcansa01,10<.<=20
198,-0.217391,4.869565,6.217391,3.000000,1.565217,5.782609,0.739130,0.173913,3.755652,19.913043,11.086957,0.478261,2023-08-18,alcansa01,10<.<=20
199,-0.166667,4.875000,6.250000,3.000000,1.541667,5.791667,0.833333,0.166667,3.763750,19.833333,11.166667,0.500000,2023-08-23,alcansa01,10<.<=20
200,-0.200000,4.840000,6.280000,3.040000,1.600000,5.680000,0.840000,0.160000,3.773600,19.840000,10.800000,0.480000,2023-08-29,alcansa01,10<.<=20


In [52]:
# Merge in all those different windows of data

# Merge in last 1 game of data
compiled_df = pd.merge(df,df_last,
                       on = ['Date','code'])

# Merge in last N games of data
compiled_df = pd.merge(compiled_df,df_lastfew,
                       on = ['Date','code'])

# Merge in all season games of data
compiled_df = pd.merge(compiled_df,df_cummulative,
                       on = ['Date','code'])

compiled_df.columns

Index(['Date', 'Opp', 'Dec', 'DR', 'H', 'ER', 'BB', 'SO', 'HR', 'HBP', 'FIP',
       'Outs', 'points', 'QS', 'pitcher', 'code', 'Dec_1', 'DR_1', 'H_1',
       'ER_1', 'BB_1', 'SO_1', 'HR_1', 'HBP_1', 'FIP_1', 'Outs_1', 'points_1',
       'QS_1', 'bucket_1', 'Dec_5', 'DR_5', 'H_5', 'ER_5', 'BB_5', 'SO_5',
       'HR_5', 'HBP_5', 'FIP_5', 'Outs_5', 'points_5', 'QS_5', 'bucket_5',
       'Dec_all', 'DR_all', 'H_all', 'ER_all', 'BB_all', 'SO_all', 'HR_all',
       'HBP_all', 'FIP_all', 'Outs_all', 'points_all', 'QS_all', 'bucket_all'],
      dtype='object')

In [54]:
# Drop the data from that day, so we're only training on data that is known at the time before the given date
training_data = compiled_df.drop(columns = [
    'Dec', 'DR', 'H', 'ER', 'BB', 'SO',
    'HR', 'HBP', 'FIP', 'Outs', 'QS'])

# Drop NaN rows
# These rows result from asking for performance on the last 1, N games when there have been fewer than 1, N games
training_data = training_data.dropna()


In [55]:
training_data.to_csv('pitcher_previous_perf.csv',index = False)

In [56]:
training_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 195 entries, 1 to 201
Data columns (total 31 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Date      195 non-null    object 
 1   Opp       195 non-null    object 
 2   points    195 non-null    int64  
 3   pitcher   195 non-null    object 
 4   code      195 non-null    object 
 5   Dec_1     195 non-null    float64
 6   DR_1      195 non-null    float64
 7   H_1       195 non-null    float64
 8   ER_1      195 non-null    float64
 9   BB_1      195 non-null    float64
 10  SO_1      195 non-null    float64
 11  HR_1      195 non-null    float64
 12  HBP_1     195 non-null    float64
 13  FIP_1     195 non-null    float64
 14  Outs_1    195 non-null    float64
 15  points_1  195 non-null    float64
 16  QS_1      195 non-null    float64
 17  bucket_1  195 non-null    object 
 18  Dec_5     195 non-null    float64
 19  DR_5      195 non-null    float64
 20  H_5       195 non-null    float6